Tom Louwerse stores the raw simulation data that is used to produce the [Peilingwijzer](https://peilingwijzer.tomlouwerse.nl) in a file that can be downloaded via the following dynamic link:

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv("https://d1bjgq97if6urz.cloudfront.net/Public/Peilingwijzer/Last/coa_seats.csv",
                   index_col=0, header=0)

Colors picked from the Peilingwijzer's colors to make the correspondence clearer:

In [ ]:
colors = {
    "VVD": "#455493",
    "PVV": "#00B9FF",
    "CDA": "#00894B",
    "D66": "#4AAB2D",
    "GL": "#006B39",
    "SP": "#C73D77",
    "PvdA": "#9A0D1B",
    "CU": "#0094B4",
    "PvdD": "#EBC30A",
    "50PLUS": "#C2791E",
    "SGP": "#7F8084",
    "Denk": "#41BAC1",
    "FvD": "#6E0C13",
    "PvdT": "#F9E518"
}

In [ ]:
data.max().max()

In [ ]:
hist_kwargs = dict(
#                alpha=0.8,
               stacked=True,
#                histtype='stepfilled',
#                density=True,
               figsize=(15,10)
)

In [ ]:
data.plot.hist(**hist_kwargs, bins=data.max().max() + 1, color=colors)

If you want to visualize coalitions in this way, you have to add up the counts per simulation and then visualize those numbers. For instance, let's take the VVD, PVV, CDA coalition, which at this point in time (10 November 2020) could form a majority, according to the sum of the polls' best estimates as published on the Peilingwijzer graph. And we compare to a huge left-wing-ish coalition.

In [ ]:
coalitions = pd.DataFrame()

In [ ]:
def sum_coalition(data, *parties):
    return sum(data[party] for party in parties)

In [ ]:
def add_coalition(coalitions_df, data, *parties, inplace=False):
    name = "+".join(parties)
    if not inplace:
        coalitions_df = coalitions_df.copy()
    coalitions_df[name] = sum_coalition(data, *parties)
    return coalitions_df

In [ ]:
coalitions = add_coalition(coalitions, data, "VVD", "PVV", "CDA")

In [ ]:
coalitions = add_coalition(coalitions, data, "GL", "SP", "PvdA", "PvdD", "Denk", "50PLUS")

In [ ]:
coalitions

In [ ]:
coalitions.plot.hist(
    **hist_kwargs,
    bins=coalitions.max().max() + 1
)

Now we have to still extract 95% confidence intervals. We can approximate by using mean and twice the standard deviation. Then round up, as is done in Peilingwijzer as well. Let's try it out for a few parties and see whether it matches.

In [ ]:
import pickle
from collections import namedtuple
Peiling = namedtuple('Peiling', ['verwacht', 'laag', 'hoog'])
with open('peilingen.pkl', 'rb') as fh:
    numbers = pickle.load(fh)

In [ ]:
import math

In [ ]:
def compare_estimator_1(party):
    est = data[party].mean().round()
    interval = math.ceil((2 * data[party].std()))
    our_estimate = Peiling(verwacht=est, laag=max(0, est - interval), hoog=min(est + interval, 150))
    correct = numbers[party] == our_estimate
    return correct, {"correct": numbers[party], "ours": our_estimate, "mean": data[party].mean(), "2 x std": 2 * data[party].std()}

In [ ]:
{party: compare_estimator_1(party) for party in data.columns}

Ok, not completely there yet, then, but pretty close. Had to add clip below 0 (and, to be technically correct, above 150).

But then still there's a few things going wrong:

- Denk is not at the mean. Is the peak instead determined at the maximum probability peak?
- GL, CU and 50PLUS have a broader confidence interval in our estimate. Perhaps ceil is not the best rounding function. We could fiddle a bit with a weird rounder that rounds up above 0.25 and down below that, or some other value that fits the distributions best.

This is probably all due to the gaussian approximation we make here. The actual model is not a Gaussian, so mean and std are flawed estimators of the true confidence interval. Anyway, as long as it's close, we can always try the actual model later.

Let's first try to compare max likelihood peak:

In [ ]:
data["Denk"].value_counts().index[0]

In [ ]:
def compare_estimator_2():
    comparison = {}

    def estimate_Peiling(est, std):
        interval = math.ceil((2 * std))
        return Peiling(verwacht=est, laag=max(0, est - interval), hoog=min(est + interval, 150))

    def check_correctness(theirs, ours):
        correct = theirs == ours
        return correct, {"correct": theirs,
                         "ours": ours,
                         "mean": data[party].mean(),
                         "2 x std": 2 * data[party].std()}

    def compare_party(party):
        est = data[party].value_counts().index[0]
        our_estimate = estimate_Peiling(est, data[party].std())
        return check_correctness(numbers[party], our_estimate)

    for party in data.columns:
        comparison[party] = compare_party(party)

    # add missing seats if necessary
    while sum(ding[1]['ours'].verwacht for ding in comparison.values()) < 150:
        rest_values = {party: thing[1]["mean"] - thing[1]["ours"].verwacht for party, thing in comparison.items()}
        party_max_rest = max(rest_values.keys(), key=(lambda k: rest_values[k]))

        ours_new = estimate_Peiling(comparison[party_max_rest][1]["ours"].verwacht + 1,
                                    comparison[party_max_rest][1]["2 x std"] / 2)

        comparison[party_max_rest] = check_correctness(numbers[party_max_rest], ours_new)

    return comparison

In [ ]:
comparison_2 = compare_estimator_2()
# comparison_2
{party: ding[0] for party, ding in comparison_2.items()}

In [ ]:
sum(ding[1]['correct'].verwacht for ding in comparison_2.values())

In [ ]:
sum(ding[1]['ours'].verwacht for ding in comparison_2.values())

Ah, ok, so Denk has to get a rest seat here, it seems.

Then, let's try the rounding fiddling to get the rest correct...

In [ ]:
def compare_estimator_3(round_dec=0.15):
    comparison = {}

    def estimate_Peiling(est, std, round_dec=round_dec):
        two_std = 2 * std
        two_std_floor = math.floor(two_std)
        two_std_rest = two_std - two_std_floor
        if two_std_rest < round_dec:
            interval = two_std_floor
        else:
            interval = math.ceil(two_std)
        return Peiling(verwacht=est, laag=max(0, est - interval), hoog=min(est + interval, 150))

    def check_correctness(theirs, ours):
        correct = theirs == ours
        return correct, {"correct": theirs,
                         "ours": ours,
                         "mean": data[party].mean(),
                         "2 x std": 2 * data[party].std()}

    def compare_party(party):
        # max likelihood or mean makes no difference in practice
#         est = data[party].value_counts().index[0]
        est = data[party].mean().round()
        our_estimate = estimate_Peiling(est, data[party].std())
        return check_correctness(numbers[party], our_estimate)

    for party in data.columns:
        comparison[party] = compare_party(party)

    # add missing seats if necessary
    while sum(ding[1]['ours'].verwacht for ding in comparison.values()) < 150:
        rest_values = {party: thing[1]["mean"] - thing[1]["ours"].verwacht for party, thing in comparison.items()}
        party_max_rest = max(rest_values.keys(), key=(lambda k: rest_values[k]))

        ours_new = estimate_Peiling(comparison[party_max_rest][1]["ours"].verwacht + 1,
                                    comparison[party_max_rest][1]["2 x std"] / 2)

        comparison[party_max_rest] = check_correctness(numbers[party_max_rest], ours_new)

    return comparison

In [ ]:
comparison_3 = compare_estimator_3()
{party: ding[0] for party, ding in comparison_3.items()}

In [ ]:
comparison_3['GL']

Ok, so this is still not perfect, but it's the closest we can get, with only one wrong fit.

On the other hand, maybe just using the gaussian approximation with rounding up is a safer way to go. We will have some wider uncertainty estimates, but we can take those to represent the fact that we do not actually use the right model.

So, let's go with estimation method 2 for the coalitions. Except, instead of max likelihood, we go back to mean, because it makes no difference in practice, but is unambiguous in case there are two equally likely bins in the histogram.

Also, let's do it with Python standard library stuff now to prepare for using it from Heroku with minimal dependencies.

In [ ]:
import statistics

In [ ]:
def to_Peiling_from_simulations(simulations):
    def estimate_Peiling(est, std):
        interval = math.ceil((2 * std))
        return Peiling(verwacht=int(est),
                       laag=int(max(0, est - interval)),
                       hoog=int(min(est + interval, 150)))

    est = round(statistics.mean(simulations))
    return estimate_Peiling(est, statistics.stdev(simulations))

In [ ]:
{party: to_Peiling_from_simulations(data[party]) == numbers[party] for party in data.columns}

In [ ]:
to_Peiling_from_simulations(coalitions["GL+SP+PvdA+PvdD+Denk+50PLUS"])

In [ ]:
to_Peiling_from_simulations(coalitions["VVD+PVV+CDA"])

That was with a Pandas Series, but we can now also do it directly on a regular list:

In [ ]:
to_Peiling_from_simulations(list(coalitions["VVD+PVV+CDA"]))

Now, when we do it for all parties, we should get a vanishing interval.

In [ ]:
coalitions = add_coalition(coalitions, data, *data.columns)

In [ ]:
coalitions.keys()

In [ ]:
to_Peiling_from_simulations(list(coalitions["VVD+PVV+CDA+D66+GL+SP+PvdA+CU+PvdD+50PLUS+SGP+Denk+FvD+PvdT"]))

Cool!

In [ ]:
to_Peiling_from_simulations(sum_coalition(data, *data.columns[:-1]))

Indeed, without PvdT (last in the list), which is estimated at 0, but could get 1, this is what you expect.

In [ ]:
to_Peiling_from_simulations(sum_coalition(data, *data.columns[:-2]))

In [ ]:
to_Peiling_from_simulations(data['FvD'])

FvD is second to last, so excluding it should indeed lower the estimate by 6. We see that the uncertainty of PvdT alone is no longer just "added" as we did before in the first Coalitiewijzer, but they are now combined into an uncertainty estimate that is apparently the same as that of FvD alone.

Time to build this into the Coalitiewijzer!

Save in native format:

In [ ]:
import pickle

In [ ]:
sims_df = pd.read_csv("https://d1bjgq97if6urz.cloudfront.net/Public/Peilingwijzer/Last/coa_seats.csv",
                       index_col=0, header=0)
sims = {party: tuple(sims_df[party]) for party in sims_df}

In [ ]:
with open('simulations.pkl', 'wb') as fh:
    pickle.dump(sims, fh)